#### Library import

In [ ]:
from functions import A_preprocessing as preprocessing
from functions import B_create_feature_store as feature_store
from functions import C_task1_model_train as create_model

## 1. Pre-processing

In [ ]:
df = preprocessing.read_dataset("data/raw_dataset/full_data_flightdelay.csv")

In [ ]:
#df.head()

In [ ]:
preprocessing.remove_null_values(df)

In [ ]:
#preprocessing.delayed_flights(df)

In [ ]:
preprocessing.create_part_of_day_column(df)
#preprocessing.show_heatmap_part_of_day(df)

In [ ]:
#df.head()

## 2. Hopsworks connection

In [ ]:
#import pandas as pd


#fs = feature_store.connect_to_hopsworks()

## 3. Task 1

In [ ]:

# # Neural Network model
# X_train, X_test, y_train, y_test = preprocessing.prepare_data_for_ML_model(df, True)

# nn_model = create_model.define_NN_model(X_train)
# history = create_model.train_NN_model(nn_model, X_train, X_test, y_train, y_test)
# create_model.show_NN_performance(history)

# MLPClassifier model
X_train, X_test, y_train, y_test = preprocessing.prepare_data_for_ML_model(df)
mlp_mode = create_model.define_MLP_classifier()
train_losses, test_losses, train_accuracies, test_accuracies = create_model.train_MLP_classifier(mlp_mode, X_train, X_test, y_train, y_test)
create_model.show_MLP_performance(train_losses, test_losses, train_accuracies, test_accuracies)


In [ ]:
create_model.save_model(mlp_mode, "modello")
